In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname="C:\Windows\Fonts\\malgun.ttf").get_name()
plt.rc("font", family=font_name)
import matplotlib as mlp
mlp.rcParams["axes.unicode_minus"] = False

from datetime import datetime
import cv2
import zipfile
import shutil
import os
import glob
import json
from tqdm.notebook import tqdm

# 견적서 CSV에 연식, 색상 추가

### 1) 사고 유형 json 가져오기

In [56]:
car_repair1 = pd.read_csv("car_repair1.csv")
car_repair2 = pd.read_csv("car_repair2.csv")
car_repair3 = pd.read_csv("car_repair3.csv")

In [57]:
print(len(car_repair1))
print(len(car_repair2))
print(len(car_repair3))

600000
637503
562713


In [58]:
all_data = pd.concat([car_repair1, car_repair2, car_repair3], ignore_index=True)
all_data.sort_values("category_id")
all_data.head()

,image_id,category_id,segmentation,area,bbox,damage,year,color,repair,supercategory_name
0,2,as-0036229,"[[[[438, 504], [440, 443], [436, 419], [439, 4...",14977.0,"[336, 159, 198, 345]",Breakage,2020,Black,"['Rear bumper:coating,exchange']",Full-size car
1,3,as-0036229,"[[[[469, 497], [448, 479], [458, 437], [461, 4...",7186.5,"[378, 209, 154, 288]",Breakage,2020,Black,"['Rear bumper:coating,exchange']",Full-size car
2,4,as-0036229,"[[[[73, 596], [69, 588], [67, 582], [65, 575],...",749.5,"[53, 543, 31, 53]",Breakage,2020,Black,"['Rear bumper:coating,exchange']",Full-size car
3,4,as-0036229,"[[[[107, 371], [108, 362], [445, 361], [795, 3...",13872.5,"[107, 356, 688, 29]",Scratched,2020,Black,"['Rear bumper:coating,exchange']",Full-size car
4,5,as-0036229,"[[[[77, 390], [87, 375], [452, 351], [560, 356...",22816.0,"[77, 351, 690, 52]",Scratched,2020,Black,"['Rear bumper:coating,exchange']",Full-size car


In [59]:
len(all_data)

1800216

### 2) 외제차 제외하기

In [60]:
kor_index = pd.read_csv("kor_car.csv")
kor_index.rename(columns = {"file_name" : 'category_id'}, inplace = True)

In [61]:
merge_car = pd.merge(all_data, kor_index, how='inner',on='category_id')

In [62]:
len(merge_car)

1393019

### 3) 필요한 컬럼만 추출

In [63]:
merge_car = merge_car[["category_id", "year", "color"]]
merge_car.head()

,category_id,year,color
0,as-0036229,2020,Black
1,as-0036229,2020,Black
2,as-0036229,2020,Black
3,as-0036229,2020,Black
4,as-0036229,2020,Black


In [66]:
merge_car["color"].value_counts()

White        930292
Black        237200
Gray tone    106640
Others        54273
Blue tone     44285
Red tone      20329
Name: color, dtype: int64

### 4) 중복된 행 제거

In [68]:
car_data = merge_car.drop_duplicates(keep = 'first', ignore_index=True)

In [67]:
#### Before ####
merge_car[merge_car["category_id"]=="as-0000025"]

,category_id,year,color
971883,as-0000025,2016,White
971884,as-0000025,2016,White
971885,as-0000025,2016,White
971886,as-0000025,2016,White
971887,as-0000025,2016,White
971888,as-0000025,2016,White
971889,as-0000025,2016,White
971890,as-0000025,2016,White
971891,as-0000025,2016,White
971892,as-0000025,2016,White


In [69]:
#### After ####
car_data[car_data["category_id"]=="as-0000025"]

,category_id,year,color
72241,as-0000025,2016,White


### 5) 견적서에 사고 정보에서 color, year 추가

In [4]:
repair_price = pd.read_csv("data/repair_price.csv")
repair_price = repair_price[["category_id", "coating", "sheet_metal", "exchange", "total"]]
repair_price.head()

,category_id,coating,sheet_metal,exchange,total
0,as-0000025,488500,175000,145750,1121128
1,as-0000025,488500,175000,145750,1121128
2,as-0000025,488500,175000,145750,1121128
3,as-0000027,496500,262500,267750,1293228
4,as-0000027,496500,262500,267750,1293228


In [75]:
data = pd.merge(repair_price, car_data, how='inner',on='category_id')

In [77]:
data = data.drop_duplicates(keep = 'first', ignore_index=True)

In [78]:
data.to_csv("color_year_data.csv", index=False)